In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Note: Our dataset is custom.

You MUST ensure that you have properly downloaded the dataset and placed it in your google drive at the base level. You cannot simply read the dataset as a "shared folder" rather, you must first download it locally then reupload so that it is stored in your google drive. Please be concious of this, otherwise erranous output might be misinterpreted as an issue with our code when in reality is an issue with how the dataset has been set up on your google drive. You can also try adding a shortcut to the dataset to your personal drive which may work but it is only garunteed if the full dataset is stored in your drive  as those are the conditions under which we trained our model.

## Subnote

If you get a "values unpack" error (something along those lines) it is due to incorrect setup of our dataset in your google drive. It is not due to an error in our code. To correct this error, ensure that the steps in the above paragraph were followed to a tee. Please follow up with us if you have questions.


# Note Two: Training takes a longgggg time.

You might notice that we only ran our model for two epochs. This is because training is very very slow, even on A100 GPU's. We are broke college students and cannot continue to purchase colab compute units, so we cut things off once we saw performance that was good enough to meet our project proposal requirements.

In [ ]:
# FINAL RUN!!!!!

#!/usr/bin/env python3
import os
import glob
import random
from collections import deque, Counter
import random

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import (
    GlobalAveragePooling2D, Dense, Input,
    TimeDistributed, LSTM, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (
    EarlyStopping, LambdaCallback,
    ReduceLROnPlateau
)
from tensorflow.keras.regularizers import l2
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

# 3. Params
base_path        = "/content/drive/MyDrive/drone_datasets/final_data"
clip_length      = 16
clip_stride      = 4      # 75% overlap
batch_size       = 8
file_seed        = 42
clip_shuffle     = 500
learning_rate    = 1e-3
max_epochs       = 2      # you can still control # of epochs, we set to 2 for now to not waste all of our compute, this is a very compute intensive model!!
dropout_rate     = 0.5
recurrent_do     = 0.1
lstm_units       = 64
subset_fraction  = 0.1    # fraction of data to use per epoch, 0.1 to minimze compute waste

# 4. Gather all files & labels
def list_videos(subdir):
    exts = ('avi','mp4','mov')
    files = []
    folder = os.path.join(base_path, subdir)
    for e in exts:
        files += glob.glob(os.path.join(folder, '**', f'*.{e}'),
                           recursive=True)
    return files

wave_files     = list_videos("wave")
not_wave_files = list_videos("not_wave")
all_files      = [(p,1) for p in wave_files] + [(p,0) for p in not_wave_files]
random.seed(file_seed)
random.shuffle(all_files)

# 5. Split file-level 73/15/15
n        = len(all_files)
n_train  = int(0.70 * n)
n_val    = int(0.15 * n)
train_files = all_files[:n_train]
val_files   = all_files[n_train:n_train+n_val]
test_files  = all_files[n_train+n_val:]

print("Raw train distribution:", Counter(l for _,l in train_files))

# 6. Oversample positives (if needed)
pos = [f for f in train_files if f[1]==1]
neg = [f for f in train_files if f[1]==0]
if pos:
    repeat_factor = len(neg) // len(pos)
    train_files = neg + pos * repeat_factor
    random.shuffle(train_files)
    print("Balanced train distribution:", Counter(l for _,l in train_files))
else:
    print("Warning: no positive samples in training set!")

# 7. Compute total_train_clips for steps_per_epoch (and subset)
#total_train_clips = 0
#for path, _ in train_files:
#    cap = cv2.VideoCapture(path)
#    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#    cap.release()
#    clips = max(0, (frames - clip_length)//clip_stride + 1)
#    total_train_clips += clips

#full_steps_per_epoch = max(1, total_train_clips // batch_size)
#steps_per_epoch = max(1, int(full_steps_per_epoch * subset_fraction))
#print(f"Total train clips: {total_train_clips}, full steps/epoch: {full_steps_per_epoch},")
#print(f"→ using {steps_per_epoch} steps ({subset_fraction*100:.1f}% of data) per epoch")

# 7. Compute total_train_clips for steps_per_epoch (and subset), hasty route (for optimized speed)
# we only open 100 files, not all of them. its good enough since we just need an approximate schedule.
# if we want to be incredibly precise and use the entire dataset, uncomment the above code.
sample_size = min(100, len(train_files))
sampled     = random.sample(train_files, sample_size)

clip_counts = []
for path, _ in sampled:
    cap = cv2.VideoCapture(path)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    clip_counts.append(max(0, (frames - clip_length)//clip_stride + 1))

avg_clips = sum(clip_counts) / len(clip_counts)
total_train_clips = int(avg_clips * len(train_files))

full_steps_per_epoch = max(1, total_train_clips // batch_size)
steps_per_epoch      = max(1, int(full_steps_per_epoch * subset_fraction))
print(f"Estimated total clips: {total_train_clips}")
print(f"→ using {steps_per_epoch} steps ({subset_fraction*100:.1f}% of data) per epoch")

# end of custom optimized step 7.

# 8. Clip-extraction helper
def extract_clips(path):
    cap = cv2.VideoCapture(path)
    buf, clips, idx = deque(maxlen=clip_length), [], 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224,224))
        buf.append(frame)
        idx += 1
        if idx >= clip_length and (idx - clip_length) % clip_stride == 0:
            clips.append(np.array(buf, dtype=np.float32))
    cap.release()
    if clips:
        return preprocess_input(np.stack(clips, axis=0))
    else:
        return np.zeros((0,clip_length,224,224,3), dtype=np.float32)

# 9. TF loader + unbatch
def load_and_split(path, label):
    def _py(p, l):
        p_str = p.numpy().decode('utf-8')
        clips_np = extract_clips(p_str)
        labels_np = np.full((clips_np.shape[0],), l.numpy(), np.int32)
        return clips_np, labels_np

    clips, lbls = tf.py_function(
        func=_py,
        inp=[path, label],
        Tout=[tf.float32, tf.int32]
    )
    clips.set_shape([None, clip_length,224,224,3])
    lbls.set_shape([None])
    return clips, lbls

def make_clip_dataset(files):
    paths, labels = zip(*files)
    ds = tf.data.Dataset.from_tensor_slices((list(paths), list(labels)))
    return ds.map(load_and_split, num_parallel_calls=tf.data.AUTOTUNE).unbatch()

train_ds = (
    make_clip_dataset(train_files)
    .shuffle(clip_shuffle, seed=file_seed)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
    .repeat()
)

val_ds = (
    make_clip_dataset(val_files)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = (
    make_clip_dataset(test_files)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# 10. Build model with updated regularization
cnn_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
cnn_base.trainable = False

inp = Input((clip_length,224,224,3))
x   = TimeDistributed(cnn_base)(inp)
x   = TimeDistributed(GlobalAveragePooling2D())(x)
x   = Dropout(dropout_rate)(x)
x   = LSTM(
    lstm_units,
    dropout=dropout_rate,
    recurrent_dropout=recurrent_do,
    kernel_regularizer=l2(1e-4),
    recurrent_regularizer=l2(1e-4)
)(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy','precision','recall','auc']
)
model.summary()

# 11. Callbacks
class F1Metrics(tf.keras.callbacks.Callback):
    def __init__(self, val_ds):
        super().__init__()
        self.val_ds = val_ds
    def on_epoch_end(self, epoch, logs=None):
        y_true, y_pred = [], []
        for Xb, yb in self.val_ds:
            probs = self.model.predict(Xb, verbose=0).flatten()
            preds = (probs > 0.5).astype(int)
            y_true.extend(yb.numpy().tolist())
            y_pred.extend(preds.tolist())
        f1 = f1_score(y_true, y_pred)
        print(f" — val_f1: {f1:.4f}")
        logs['val_f1'] = f1

print_cb   = LambdaCallback(on_epoch_end=lambda e, logs:
    print(f"\nEpoch {e+1}: loss={logs['loss']:.4f}, val_auc={logs['val_auc']:.4f}", end='')
)
f1_cb      = F1Metrics(val_ds)
lr_cb      = ReduceLROnPlateau(monitor='val_f1', factor=0.5, patience=3, mode='max', verbose=1)
early_stop = EarlyStopping(monitor='val_f1', mode='max', patience=3, restore_best_weights=True)

class_weight = {
  0: 1.0,
  1: len(neg) / len(pos)   # adjust as before
}

# 12. Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=max_epochs,
    steps_per_epoch=steps_per_epoch,
    class_weight=class_weight,
    callbacks=[print_cb, f1_cb, lr_cb, early_stop],
    verbose=1
)

# 13. Plot Loss & Validation F1
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history.get('val_loss', []), label='val_loss')
plt.title('Loss'); plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history.get('val_f1', []), label='val_f1')
plt.title('Validation F1'); plt.legend()
plt.show()

# 14. Evaluate on test set
print("\nBatch-level evaluation:")
model.evaluate(test_ds, verbose=1)

# 15. Per-video accuracy
video_results = []
for path, label in test_files:
    clips = extract_clips(path)
    if clips.shape[0] == 0: continue
    preds = model.predict(clips, batch_size=batch_size, verbose=0).flatten()
    avg_p = preds.mean()
    pred_label = int(avg_p > 0.5)
    video_results.append((path, label, avg_p, pred_label))

n_vid = len(video_results)
n_corr = sum(1 for _,lbl,_,pred in video_results if pred==lbl)
print(f"\nPer-video accuracy on {n_vid} videos: {n_corr/n_vid:.4f}")

# 16. Save final model
save_path = os.path.join(base_path, 'wave_sequence_model_final5.keras')
model.save(save_path)
print(f"Saved model to {save_path}")